# Detection of other arm activities
This notebook shows how to predict the probability of other arm activities during gait using extracted features from windowed accelerometer and gyroscope data as input.

## Modules

In [16]:
import numpy as np
import os
import pandas as pd
import tsdf

from dbpd import DataColumns

## Constants

In [17]:
path_to_data_folder = '../../../tests/data'
path_to_classifier_input = os.path.join(path_to_data_folder, '0.classifiers')
path_to_input_data = os.path.join(path_to_data_folder, '3.extracted_features')
path_output = os.path.join(path_to_data_folder, '4.predictions')

classifier_file_name = 'asd_classifier.pkl'

meta_filename = 'arm_swing_meta.json'
time_filename = 'arm_swing_time.bin'
values_filename = 'arm_swing_values.bin'

l_accel_cols = [DataColumns.ACCELERATION_X, DataColumns.ACCELERATION_Y, DataColumns.ACCELERATION_Z]
l_gyro_cols = [DataColumns.ROTATION_X, DataColumns.ROTATION_Y, DataColumns.ROTATION_Z]

## Load data

#### Features

In [18]:
metadata_dict = tsdf.load_metadata_from_path(os.path.join(path_to_input_data, meta_filename))
metadata_time = metadata_dict[time_filename]
metadata_samples = metadata_dict[values_filename]

df = tsdf.load_dataframe_from_binaries([metadata_time, metadata_samples], tsdf.constants.ConcatenationType.columns)

df.head(2)

,time,angle_perc_power,range_of_motion,forward_peak_ang_vel_mean,forward_peak_ang_vel_std,backward_peak_ang_vel_mean,backward_peak_ang_vel_std,std_norm_acc,grav_acceleration_x_mean,grav_acceleration_x_std,...,cc_7_rotation,cc_8_rotation,cc_9_rotation,cc_10_rotation,cc_11_rotation,cc_12_rotation,cc_13_rotation,cc_14_rotation,cc_15_rotation,cc_16_rotation
0,0.00,0.937852,2.875672,8.193249,3.292503,4.938885,0.0,0.154038,-0.255275,0.201556,...,0.914803,3.981584,2.594423,2.683692,1.498986,0.654959,-1.101154,0.551201,-0.356018,0.606395
1,0.75,0.992215,4.295439,3.288568,1.612178,4.401343,0.0,0.160067,-0.396332,0.176710,...,2.565799,4.660158,1.949670,2.904717,1.364006,-0.064123,-2.094619,0.494000,1.354037,1.760634


#### Classifier

In [19]:
clf = pd.read_pickle(os.path.join(path_to_classifier_input, classifier_file_name))

## Prepare data

In [20]:
clf.feature_names_in_ = [f'{x}_power_below_gait' for x in l_accel_cols] + \
                        [f'{x}_power_gait' for x in l_accel_cols] + \
                        [f'{x}_power_tremor' for x in l_accel_cols] + \
                        [f'{x}_power_above_tremor' for x in l_accel_cols] + \
                        [f'grav_{x}_{y}' for x in l_accel_cols for y in ['mean', 'std']] + \
                        ['std_norm_acc'] + [f'cc_{i}_acceleration' for i in range(1, 17)] + \
                        [f'{x}_dominant_frequency' for x in l_accel_cols] + \
                        [f'cc_{i}_rotation' for i in range(1, 17)] + \
                        [f'{x}_dominant_frequency' for x in l_gyro_cols] + \
                        ['range_of_motion', 'forward_peak_ang_vel_mean',
                         'forward_peak_ang_vel_std', 'backward_peak_ang_vel_mean',
                         'backward_peak_ang_vel_std', 'angle_perc_power']

X = df.loc[:, clf.feature_names_in_]

## Predict 

In [21]:
df['pred_arm_swing_proba'] = clf.predict_proba(X)[:, 1]
df['pred_arm_swing'] = clf.predict(X)

## Store predictions

In [22]:
metadata_samples.__setattr__('file_name', 'arm_swing_values.bin')
metadata_samples.__setattr__('file_dir_path', path_output)
metadata_time.__setattr__('file_name', 'arm_swing_time.bin')
metadata_time.__setattr__('file_dir_path', path_output)

metadata_samples.__setattr__('channels', ['pred_arm_swing_proba'])
metadata_samples.__setattr__('units', ['probability'])

metadata_time.__setattr__('channels', ['time'])
metadata_time.__setattr__('units', ['s'])
metadata_time.__setattr__('data_type', np.int64)

In [23]:
if not os.path.exists(path_output):
    os.makedirs(path_output)

# store binaries and metadata
tsdf.write_dataframe_to_binaries(path_output, df, [metadata_time, metadata_samples])
tsdf.write_metadata([metadata_time, metadata_samples], 'arm_swing_meta.json')